In [ ]:
# %matplotlib widget
import numpy as np
import glob
import torch
import torch.nn as nn
from skimage.measure import label
from skimage.segmentation import find_boundaries
from torch.utils.data import DataLoader
# from ipywidgets import interact
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import os
import folium
import matplotlib.colors as mcolors
import unfoldNd
import itertools


from model.mlp import MLP
from tools.train_double_logistic import double_logistic_function
from utils.helpers import get_doy, check_missing_timestamps

# from data.patchwise_dataset import PatchwiseDataset

In [ ]:
# file_path = "/Volumes/Macintosh HD/Users/davidbruggemann/OneDrive/DIMPEO/data/tmp9_train.h5"

# ds = PatchwiseDataset(
#         file_path,
#         pixelwise=False,
#         annual=False,
#         spatiotemporal_features=["s2_ndvi", "s2_B04", "s2_B03", "s2_B02"],
#         spatial_features=["valid_mask", "drought_mask"],
#     )

# torch.manual_seed(1)
# loader = DataLoader(ds, batch_size=1, drop_last=True, shuffle=True)
# loader_iter = iter(loader)


In [ ]:
OUTLIER_FACTOR = 1.5
OUTLIER_FACTOR_NN = 1.5
MAX_MISSINGNESS = 0.95
COUNT_THRESHOLD = 100
WINDOW_SIZE = 3
T_WINDOW_SIZE = 5
NUM_YEARS = 7
NUM_DATAPOINTS_PER_YEAR = 73
H = 128
W = 128
START_YEAR = 2017
END_YEAR = 2023

In [ ]:
# plt.rcParams["figure.figsize"] = (16,8)

# def browse_images(rgb, ndvi, l_bound, h_bound):
#     N = rgb.shape[0]
#     ndvi_vmin = ndvi[~torch.isnan(ndvi)].min()
#     ndvi_vmax = ndvi[~torch.isnan(ndvi)].max()
#     fig, ax = plt.subplots(1, 2, sharey=True)
#     im1 = ax[0].imshow(rgb[0])
#     im2 = ax[0].imshow(l_bound[0, :, :], cmap=matplotlib.colors.ListedColormap(['white', 'red']), alpha=1.0 * (l_bound[0, :, :] > 0))
#     im3 = ax[0].imshow(h_bound[0, :, :], cmap=matplotlib.colors.ListedColormap(['blue', 'white']), alpha=1.0 * (h_bound[0, :, :] > 0))
#     im4 = ax[1].imshow(ndvi[0], vmin=ndvi_vmin, vmax=ndvi_vmax)
#     im5 = ax[1].imshow(l_bound[0, :, :], cmap=matplotlib.colors.ListedColormap(['white', 'red']), alpha=1.0 * (l_bound[0, :, :] > 0))
#     im6 = ax[1].imshow(h_bound[0, :, :], cmap=matplotlib.colors.ListedColormap(['blue', 'white']), alpha=1.0 * (h_bound[0, :, :] > 0))
#     fig.colorbar(im4, ax=ax, fraction=0.046, pad=0.04)
#     fig.suptitle("red: NDVI too low, blue: NDVI too high")
#     def update(i=0):
#         im1.set_data(rgb[i])
#         im2.set_data(l_bound[i, :, :])
#         im2.set_alpha(1.0 * (l_bound[i, :, :] > 0))
#         im3.set_data(h_bound[i, :, :])
#         im3.set_alpha(1.0 * (h_bound[i, :, :] > 0))
#         im4.set_data(ndvi[i])
#         im5.set_data(l_bound[i, :, :])
#         im5.set_alpha(1.0 * (l_bound[i, :, :] > 0))
#         im6.set_data(h_bound[i, :, :])
#         im6.set_alpha(1.0 * (h_bound[i, :, :] > 0))
#         fig.canvas.draw_idle()
#     interact(update, i=(0, N-1));

# def lin_stretch_vec(band):
#     v = torch.nanquantile(band, torch.tensor((0.02, 0.98), dtype=band.dtype))
#     temp = 255 * (band - v[0]) / (v[1] - v[0])
#     temp = torch.clip(temp, 0, 255)
#     return temp

In [ ]:
def map(minicube):
    x_min, x_max, y_min, y_max = minicube.lon[0].values, minicube.lon[-1].values, minicube.lat[-1].values, minicube.lat[0].values
    # Create map
    interactive_map = folium.Map(
        location=((y_min+y_max)/2, (x_min+x_max)/2),
        zoom_start=min(int((np.log(1000 / (x_max-x_min)) / np.log(2))), int((np.log(1000 / (y_max-y_min)) / np.log(2)))),
        tiles="http://mt1.google.com/vt/lyrs=y&z={z}&x={x}&y={y}",
        attr="Google",
        prefer_canvas=True,
    )
    
    folium.TileLayer('openstreetmap', show=False).add_to(interactive_map)
    
    # Create bounding box coordinates to overlay on map
    line_segments = [
        (y_min, x_min),
        (y_min, x_max),
        (y_max, x_max),
        (y_max, x_min),
        (y_min, x_min),
    ]

    # Add bounding box as an overlay
    interactive_map.add_child(
        folium.features.PolyLine(locations=line_segments, color="black", opacity=0.8)
    )
    
    return interactive_map

In [ ]:
folder='/data_2/dimpeo/cubes'
filepaths = sorted(list(glob.glob(os.path.join(folder, "*_raw.nc"))))
device = "cuda"


def cartesian_product(a, b):
    return np.array(list(itertools.product(a, b)))


encoder = MLP(d_in=8, d_out=8, n_blocks=8, d_block=256, dropout=0, skip_connection=True).to(device)
encoder.load_state_dict(torch.load("/data_1/scratch_1/dbrueggemann/nn/encoder_noposenc_notwi.pt"))
encoder.eval()

# rescale input (stats obtained from full dataset)
# MEANS = torch.tensor([
#     8.34458,  # lon
#     46.32019,  # lat
#     1104.5027,  # dem
#     43.102547,  # fc
#     15.638395,  # fh
#     26.418936,  # slope
#     -0.0069493465,  # easting
#     0.062305786,  # northing
#     2.8649843,  # twi
#     ]).to(device).unsqueeze(1).unsqueeze(1)
# STDS = torch.tensor([
#     0.9852665,  # lon
#     0.61929077,  # lat
#     409.8894,  # dem
#     40.840992,  # fc
#     30.395235,  # fh
#     13.358144,  # slope
#     0.69276434,  # easting
#     0.7184338,  # northing
#     1.861385,  # twi
#     ]).to(device).unsqueeze(1).unsqueeze(1)

MEANS = {
    "lon": 8.34458,
    "lat": 46.32019,
    "dem": 1104.5027,
    "fc": 43.102547,
    "fh": 15.638395,
    "slope": 26.418936,
    "easting": -0.0069493465,
    "northing": 0.062305786,
    "twi": 2.8649843,
    "rugg": 8.363732,
    "curv": 6.1888146e-05,
}
STDS = {
    "lon": 0.9852665,
    "lat": 0.61929077,
    "dem": 409.8894,
    "fc": 40.840992,
    "fh": 30.395235,
    "slope": 13.358144,
    "easting": 0.69276434,
    "northing": 0.7184338,
    "twi": 1.861385,
    "rugg": 5.4581103,
    "curv": 0.0061930786,
}
features = ["lon", "lat", "dem", "fc", "fh", "slope", "easting", "northing"]
means = torch.tensor([MEANS[f] for f in features], dtype=float, device=device).unsqueeze(1).unsqueeze(1)
stds = torch.tensor([STDS[f] for f in features], dtype=float, device=device).unsqueeze(1).unsqueeze(1)

T_SCALE = 1.0 / 365.0
t_plot = torch.linspace(0, 365, 1000).unsqueeze(0) * T_SCALE
YEARS_IN_TRAIN = 6
SPLIT_IDX = 73 * YEARS_IN_TRAIN


# for path in filepaths:
while True:
    with torch.no_grad():
        rand_ix = np.random.randint(0, len(filepaths))
        path = filepaths[rand_ix]

        try:
            minicube = xr.open_dataset(path, engine="h5netcdf")
        except OSError:
            continue

        missing_dates = check_missing_timestamps(minicube, START_YEAR, END_YEAR)
        if missing_dates:
            minicube = minicube.reindex(
                time=np.sort(np.concatenate([minicube.time.values, missing_dates]))
            )

        try:
            ndvi = minicube.s2_ndvi.where((minicube.s2_mask == 0) & minicube.s2_SCL.isin([1, 2, 4, 5, 6, 7])).values
            forest_mask = (minicube.FOREST_MASK.values > 0.8)
        except AttributeError:
            continue

        # convert to torch
        ndvi = torch.from_numpy(ndvi).to(device)
        doy = torch.from_numpy(get_doy(minicube.time.values)).to(device)
        forest_mask = torch.from_numpy(forest_mask).to(device)

        # mask out non-forest pixels
        ndvi[:, ~forest_mask] = torch.nan

        # features for network
        longitude = np.array(minicube.lon.values, dtype=np.float32)
        latitude = np.array(minicube.lat.values, dtype=np.float32)
        lon_lat = cartesian_product(longitude, latitude).reshape(len(longitude), len(latitude), 2)
        lon_lat = torch.from_numpy(lon_lat).permute(2, 0, 1).to(device)
        dem = torch.from_numpy(minicube.DEM.values).unsqueeze(0).to(device)
        fc = torch.from_numpy(minicube.FC.values).unsqueeze(0).to(device)
        fh = torch.from_numpy(minicube.FH.values).unsqueeze(0).to(device)
        slope = torch.from_numpy(minicube.slope.values).unsqueeze(0).to(device)
        easting = torch.from_numpy(minicube.easting.values).unsqueeze(0).to(device)
        northing = torch.from_numpy(minicube.northing.values).unsqueeze(0).to(device)
        # twi = torch.from_numpy(minicube.twi.values).unsqueeze(0).to(device)

        fc[fc == -9999] = torch.nan
        inp = torch.cat([lon_lat, dem, fc, fh, slope, easting, northing], axis=0)  # 9 x H x W
        inp = (inp - means) / stds
        inp = torch.nan_to_num(inp, nan=0.0)

        # only forward forest pixels
        masked_inp = inp[:, forest_mask]
        masked_inp = masked_inp.permute(1, 0)  # B x 9

        preds = encoder(masked_inp.float())
        paramsl = preds[:, [0, 1, 2, 3, 4, 5]]
        paramsu = torch.cat([preds[:, [0, 1, 2, 3]], preds[:, [4, 5]] + nn.functional.softplus(preds[:, [6, 7]])], axis=1)

        # find anomalies
        doy_test = doy[SPLIT_IDX:]
        ndvi_lower_pred = double_logistic_function(doy_test.unsqueeze(0).repeat(paramsl.shape[0], 1), paramsl).permute(1, 0)
        ndvi_upper_pred = double_logistic_function(doy_test.unsqueeze(0).repeat(paramsu.shape[0], 1), paramsu).permute(1, 0)

        iqr = torch.abs(ndvi_upper_pred - ndvi_lower_pred)

        l_thresh = (ndvi_lower_pred - OUTLIER_FACTOR_NN * iqr)  # 73 x B
        h_thresh = (ndvi_upper_pred + OUTLIER_FACTOR_NN * iqr)

        l_anom_nn = torch.full((NUM_DATAPOINTS_PER_YEAR, H, W), torch.nan, device=device)
        h_anom_nn = torch.full((NUM_DATAPOINTS_PER_YEAR, H, W), torch.nan, device=device)
        ndvi_test = ndvi[SPLIT_IDX:, forest_mask]
        ndvi_nan_mask = torch.isnan(ndvi_test)
        l_anoms = (ndvi_test < l_thresh).float()
        l_anoms[ndvi_nan_mask] = torch.nan
        l_anom_nn[:, forest_mask] = l_anoms
        h_anoms = (ndvi_test > h_thresh).float()
        h_anoms[ndvi_nan_mask] = torch.nan
        h_anom_nn[:, forest_mask] = h_anoms

        # # TODO: do post-processing
        # l_anom_nn = torch.full((NUM_DATAPOINTS_PER_YEAR, H, W), torch.nan, device=device)
        # h_anom_nn = torch.full((NUM_DATAPOINTS_PER_YEAR, H, W), torch.nan, device=device)
        # diff = (l_thresh - ndvi[SPLIT_IDX:, forest_mask]) / iqr
        # diff = torch.clamp(diff, min=0)
        # # diff[torch.isnan(diff)] = 0
        # l_anom_nn[:, forest_mask] = diff.float()

        # diff = (ndvi[SPLIT_IDX:, forest_mask] - h_thresh) / iqr
        # diff = torch.clamp(diff, min=0)
        # # diff[ndvi[SPLIT_IDX:, forest_mask] <= h_thresh] = 0
        # # diff[torch.isnan(diff)] = 0
        # h_anom_nn[:, forest_mask] = diff.float()

        # l_anom_nn_sp = torch.tensor_split(l_anom_nn, 12, dim=0)
        # h_anom_nn_sp = torch.tensor_split(h_anom_nn, 12, dim=0)
        
        # l_anom_nn[:, forest_mask] = ndvi[SPLIT_IDX:, forest_mask] < l_thresh
        

        # group by day of the year
        # ndvi = (
        #     ndvi.view(NUM_YEARS, NUM_DATAPOINTS_PER_YEAR, H, W)
        #     .permute(1, 0, 2, 3)
        #     .contiguous()
        # )
        ndvi_grouped = ndvi.view(1, NUM_YEARS, NUM_DATAPOINTS_PER_YEAR, H, W)

        # separate fitting set from test
        fitset = ndvi_grouped[:, :NUM_YEARS - 1]
        testset = ndvi_grouped[:, NUM_YEARS - 1]

        padding = int((WINDOW_SIZE - 1) / 2)
        t_padding = int((T_WINDOW_SIZE - 1) / 2)
        # pad in space
        fitset_pad = nn.functional.pad(fitset, (padding, padding, padding, padding, 0, 0), "constant", torch.nan)
        # pad in time
        fitset_pad = torch.cat((fitset_pad[:, :, -t_padding:], fitset_pad, fitset_pad[:, :, :t_padding]), dim=2)

        unfold = unfoldNd.UnfoldNd(kernel_size=(T_WINDOW_SIZE, WINDOW_SIZE, WINDOW_SIZE), dilation=1, padding=0, stride=1)
        fitset_unfold = unfold(fitset_pad)
        
        valid_mask_unfold = forest_mask.view(1, 1, 1, H, W).expand(1, fitset_unfold.shape[1], NUM_DATAPOINTS_PER_YEAR, H, W).reshape(1, -1, NUM_DATAPOINTS_PER_YEAR * H * W)
        fitset_unfold[~valid_mask_unfold.bool()] = torch.nan

        # pixels with too few observations are ignored
        invalid_mask = torch.sum(torch.isnan(fitset_unfold), dim=1, keepdim=True) / fitset_unfold.shape[1] > MAX_MISSINGNESS
        fitset_unfold[invalid_mask.expand_as(fitset_unfold)] = torch.nan

        q1 = torch.nanquantile(fitset_unfold, 0.25, dim=1)
        q3 = torch.nanquantile(fitset_unfold, 0.75, dim=1)
        iqr = torch.abs(q3 - q1)

        l_thresh = (q1 - OUTLIER_FACTOR * iqr).view(NUM_DATAPOINTS_PER_YEAR, H, W)
        h_thresh = (q3 + OUTLIER_FACTOR * iqr).view(NUM_DATAPOINTS_PER_YEAR, H, W)

        l_anom = testset.squeeze() < l_thresh  # 73 x H x W
        h_anom = testset.squeeze() > h_thresh

        # l_labels, l_num = label(l_anom, background=0, connectivity=2, return_num=True)
        # for c in range(1, l_num + 1):
        #     count = np.count_nonzero(l_labels == c)
        #     if count < COUNT_THRESHOLD:
        #         l_labels[l_labels == c] = 0
        # l_labels[l_labels > 0] = 1

        # h_labels, h_num = label(h_anom, background=0, connectivity=2, return_num=True)
        # for c in range(1, h_num + 1):
        #     count = np.count_nonzero(h_labels == c)
        #     if count < COUNT_THRESHOLD:
        #         h_labels[h_labels == c] = 0
        # h_labels[h_labels > 0] = 1

        # anom = l_labels | h_labels

        # labels, num = label(anom, background=0, connectivity=2, return_num=True)

        # print("Found {} anomalies.".format(num))

        # ndvi = testset.squeeze(0)

        # red = sample["spatiotemporal"][:, -NUM_DATAPOINTS_PER_YEAR:, :, :, 1].squeeze(0)
        # green = sample["spatiotemporal"][:, -NUM_DATAPOINTS_PER_YEAR:, :, :, 2].squeeze(0)
        # blue = sample["spatiotemporal"][:, -NUM_DATAPOINTS_PER_YEAR:, :, :, 3].squeeze(0)

        # # mask out invalid
        # red[(valid_mask == 0).view(1, H, W).expand_as(red)] = torch.nan
        # green[(valid_mask == 0).view(1, H, W).expand_as(green)] = torch.nan
        # blue[(valid_mask == 0).view(1, H, W).expand_as(blue)] = torch.nan
        # red_mask = ~torch.isnan(red)
        # green_mask = ~torch.isnan(green)
        # blue_mask = ~torch.isnan(blue)
        # rgb_mask = red_mask & green_mask & blue_mask

        # # R: https://www.rdocumentation.org/packages/raster/versions/3.6-26/topics/plotRGB
        # red = lin_stretch_vec(red)
        # green = lin_stretch_vec(green)
        # blue = lin_stretch_vec(blue)
        # rgb = torch.stack((red, green, blue), dim=-1)  # T x H x W x 3
        # rgb[~rgb_mask.unsqueeze(-1).expand_as(rgb)] = torch.nan
        # rgb = torch.nan_to_num(rgb, nan=0).to(torch.uint8)

        # rgb = torch.stack((red, green, blue), dim=-1)  # T x H x W x 3
        # rgb[~rgb_mask.unsqueeze(-1).expand_as(rgb)] = torch.nan
        # rgb = rgb / 0.2  # Vitus: / 0.4  # Samantha: / 0.22
        # rgb = torch.clip(rgb, 0.0, 1.0)
        # rgb = torch.nan_to_num(rgb, nan=0)

        # l_boundaries = find_boundaries(l_labels, mode='outer').astype(float)  # T x H x W
        # h_boundaries = find_boundaries(h_labels, mode='outer').astype(float)  # T x H x W

        #
        # PLOT
        #
        ndvi_lower_plot = double_logistic_function(t_plot.repeat(paramsl.shape[0], 1), paramsl.cpu())
        ndvi_upper_plot = double_logistic_function(t_plot.repeat(paramsu.shape[0], 1), paramsu.cpu())

        masked_ndvi = ndvi[:, forest_mask]
        masked_ndvi = masked_ndvi.permute(1, 0)  # B x 7*73
        masked_ndvi_test = masked_ndvi[:, SPLIT_IDX:].cpu()
        masked_nan_mask_test = torch.isnan(masked_ndvi_test)
        

        fig, ax = plt.subplots(6, 2, figsize=(15, 18))
        random_indices = np.random.choice(np.arange(paramsl.shape[0]), size=12, replace=False)
        for pl_idx, bi in enumerate(random_indices):

            row, col = divmod(pl_idx, 2)

            iqr = ndvi_upper_plot[bi] - ndvi_lower_plot[bi]

            masked_ndvi_test_i = masked_ndvi_test[bi][~masked_nan_mask_test[bi]]
            masked_doy_test_i = doy_test[~masked_nan_mask_test[bi]].cpu()
            ax[row, col].scatter(masked_doy_test_i, masked_ndvi_test_i, label='Observed NDVI')
            ax[row, col].fill_between(t_plot[0] / T_SCALE, ndvi_lower_plot[bi], ndvi_upper_plot[bi], alpha=0.2, color='red')
            ax[row, col].plot(t_plot[0] / T_SCALE, ndvi_lower_plot[bi] - 1.5 * iqr, c='k')
            ax[row, col].plot(t_plot[0] / T_SCALE, ndvi_upper_plot[bi] + 1.5 * iqr, c='k')
            ax[row, col].set_ylabel("NDVI")
            ax[row, col].set_ylabel("DOY")
            ax[row, col].set_ylim(-0.1, 1)

        fig.show()

        break

# browse_images(rgb, ndvi, l_boundaries, h_boundaries)

In [ ]:
# outcome of the above
l_anom_nn_np = l_anom_nn.cpu().numpy()
h_anom_nn_np = h_anom_nn.cpu().numpy()

l_anom_np = l_anom.cpu().numpy()
h_anom_np = h_anom.cpu().numpy()

interactive_map = map(minicube)

bounds = [[float(minicube.lat[-1].values), float(minicube.lon[0].values)], [float(minicube.lat[0].values), float(minicube.lon[-1].values)]]

colors = [(0, 0, 0, 0), (1, 0, 0, 1), (0, 1, 0, 1)]   # transparent, R, G
cmap = mcolors.ListedColormap(colors)
norm = mcolors.BoundaryNorm([-0.5, 0.5, 1.5, 2.5], cmap.N)

colors_nn = [(0, 0, 0, 0), (1, 0.5, 0, 1), (0, 0, 1, 1)]   # transparent, Y, B
cmap_nn = mcolors.ListedColormap(colors_nn)
norm_nn = mcolors.BoundaryNorm([-0.5, 0.5, 1.5, 2.5], cmap.N)

# define a month as anomalous if >80% of valid pixels in that month are outliers

# cond_prob_months = np.array_split(cond_prob, 12, axis=0)
l_anomal_nn = np.array_split(l_anom_nn_np, 12, axis=0)
h_anomal_nn = np.array_split(h_anom_nn_np, 12, axis=0)

l_anomal = np.array_split(l_anom_np, 12, axis=0)
h_anomal = np.array_split(h_anom_np, 12, axis=0)

# anom_threshold = -5
for month_idx, (l_month, h_month, l_month_nn, h_month_nn) in enumerate(zip(l_anomal, h_anomal, l_anomal_nn, h_anomal_nn)):
    # print(month_cp.shape)
    # valid = ~np.isnan(month_cp)
    # anom = np.zeros_like(month_cp)
    # anom[valid & (np.log(month_cp) < anom_threshold)] = 1
    # anom = np.sum(anom, axis=0) / np.sum(valid, axis=0) > 0.5

    month = np.zeros((l_month.shape[1], l_month.shape[2]))
    month[np.sum(l_month.astype(int), axis=0) > 0] = 1
    month[np.sum(h_month.astype(int), axis=0) > 0] = 2

    month_nn = np.zeros((l_month_nn.shape[1], l_month_nn.shape[2]))
    # # if more than 80% are outliers
    # l_month_nn[l_month_nn > 0] = 1
    # h_month_nn[h_month_nn > 0] = 1
    month_nn[np.nanmean(l_month_nn, axis=0) > 0.8] = 1
    month_nn[np.nanmean(h_month_nn, axis=0) > 0.8] = 2
    colored_anom = cmap(norm(month))
    colored_anom_nn = cmap_nn(norm_nn(month_nn))
    # alpha = np.where(month != 0, 0.2, 0)[:, :, None]
    # rgb_anom = np.concatenate((colored_anom[..., :3], alpha), axis=2)
    # rgb_anom = colored_anom

    folium.raster_layers.ImageOverlay(
        image=colored_anom,
        bounds=bounds,
        mercator_project=True,
        name=f"Statistical anomalies in month {month_idx})",
        opacity=0.8,
    ).add_to(interactive_map)

    folium.raster_layers.ImageOverlay(
        image=colored_anom_nn,
        bounds=bounds,
        mercator_project=True,
        name=f"NN anomalies in month {month_idx})",
        opacity=0.8,
    ).add_to(interactive_map)

# for year in range(changes_ndvi.shape[1]):
#     colored_ndvi = cmap(norm(changes_ndvi[:, year, :, :].squeeze()))

#     alpha_ndvi = np.where((changes_ndvi[:, year, :, :].squeeze() != 0) & (cube.FOREST_MASK.squeeze() != 0), 1, 0)

#     rgba_ndvi = np.dstack((colored_ndvi[..., :3], alpha_ndvi))

#     folium.raster_layers.ImageOverlay(
#         image=rgba_ndvi,
#         bounds=bounds,
#         mercator_project=True,
#         name=f"NDVI change (year {year})",
#         opacity=0.9,
#     ).add_to(interactive_map)

folium.map.LayerControl(position='topright', collapsed=False, autoZIndex=True).add_to(interactive_map)

interactive_map

In [ ]:
np.nanmean(np.array([np.nan, np.nan])) > 0.8